## 계산 그래프
- 복잡한 수학적 표현식은 계산 그래프 (Computational Graphs)로 시각화할 수 있다. 
- 예를 들어, 다음과 같이 덧셈이 2개와 곱셈 1개로 이루어진 식이 있다고 할 때,

$$ e=(a+b)\times(b+1) $$

- 곱셈은 $e=c \times d$로, 덧셈은 $c=a+b$, $d=b+1$로 나타낼 수 있으므로 계산 그래프로 나타내면 다음과 같다.

<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/y/yusuke_ujitoko/20161231/20161231102417.png" width="400"></img>

- 위 식을 값을 구하려면, 입력 변수에 특정값을 할당하여 순서대로 계산한다. 
- 예를 들어, $a=2$, $b=3$을 더하면, $e=20$이 된다. 

<img src="https://cdn-ak.f.st-hatena.com/images/fotolife/y/yusuke_ujitoko/20161231/20161231102756.png" width="400"></img>

- 이와 같이 계산 그래프를 사용하면 아무리 복잡한 계산도 각 노드별로 지역 계산으로 단순화할 수 있다.

## 순전파와 역전파
- 계산 결과를 왼쪽에서 오른쪽으로 진행하는 전달하는 것을 순전파(forward propagation)라고 한다.
- 오른쪽에서 왼쪽으로 전달하는 것을 역전파(backward propagation)라고 하고 이는 연쇄법칙(chain rule)에 의해 전달된다.  
<img src="https://kratzert.github.io/images/bn_backpass/chainrule_example.PNG" width="600"></img>

- 연쇄법칙은 미분 공식에서 가장 중요한 것 중의 하나로 합성 함수의 미분은 각 함수의 곱으로 주어진다는 것이다.

> $f$와 $g$가 미분 가능하고, $ F = f \circ g $가 $F(x) = f(g(x))$로 정의된 합성함수라면, $F$는 $x$에서 미분 가능하고, $F'$는 다음과 같은 곱으로 주어진다. 

$$ F'(x) = f'(g(x)) \cdot g'(x) $$

> 이를 라이프니츠 기호로 나타내면, $y = f(u) $ 와 $ u = g(x) $가 모두 미분가능한 함수일 때 다음이 성립한다. 

$$ \frac{dy}{dx} = \frac{dy}{du}\frac{du}{dx} $$

- 따라서 연쇄법치게 따라, 다음과 같이 $x$에 대한 $L$의 미분($x$가 증가했을 때, $L$의 변화율) 은 $L$에 대한 $z$의 미분과 $z$에 대한 $x$의 미분의 곱으로 나타낼 수 있다. 

$$\frac{\partial{L}}{\partial{x}} = \frac{\partial{L}}{\partial{z}}\frac{\partial{z}}{\partial{x}} $$

# 활성화 함수
### ReLu 
- ReLU는 다음과 같다

$$
y =\begin{cases} 
    x & (x > 0) \\ 
    0 & (x \leq 0)\\
   \end{cases}
$$

- $x$에 대한 $y$의 미분은 다음과 같다. (순전파의 $x$값이 0보다 크면 그대로 전달하고, 0 이하면 전달하지 않는다.)
$$
\frac{\partial{y}}{\partial{x}} = \begin{cases} 
    1 & (x > 0) \\ 
    0 & (x \leq 0)\\
   \end{cases}
$$

- 구현 예제이다.

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None # 요소의 값이 0보다 작거나 같으면 True, 아니면 False로 셋팅하는 마스크이다.
        
    def forward(self, x):
        self.mask = (x <= 0) 
        out = x.copy() 
        out[self.mask] = 0 # 
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

### Sigmoid
- 시그모이드는 다음과 같다.
$$ y = \frac{1}{(1 + e^{-x})} $$

- $y = \frac{1}{u}$ 이고, $ u = 1 + e^{-x} $라고 하면, $f'(u) = - \frac{1}{u^2}$이고, $ g'(x) = (-1) \cdot e^{-x}$이므로,

- 시그모이드 함수의 미분은 다음과 같이 전개할 수 있다.

$$
\begin{split}
F'(x) & = - \frac{1}{{\left(1 + e^{-x}\right)}^2} \cdot -e^{-x} \\
& = \frac{e^{-x}}{{\left(1 + e^{-x}\right)}^2} \\
& = \frac{e^{-x} + 1 - 1}{{\left(1 + e^{-x}\right)}^2} \\
& = \frac{1 + e^{-x}}{{\left(1 + e^{-x}\right)}^2} - {\left(\frac{1}{1 + e^{-x}}\right)}^2 \\
& = \frac{1}{1 + e^{-x}} - {\left(\frac{1}{1 + e^{-x}}\right)}^2 \\
& = y - y^2 \\
& = y(1-y)
\end{split}
$$

- 즉, 시그모이드의 역전파는 출력만으로 계산할 수 있다.
- 구현 예는 다음과 같다.

In [1]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, out):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## Affine/Softmax
### Affine
- 순전파 시에 신경망 출력, $f(\textbf{W}x+\textbf{b})$을 구하기 위해 행렬의 내적을 사용했다. 이를 기하학에서는 **아핀 변환(affine transformation)**이라고 한다.

- 아핀 변환 $\textbf{X} \bullet \textbf{W}$에 역전파는 다음과 같다. (행렬에 대한 입력 데이터 X가 N개의 배치인 경우도 동일)
$$ \frac{\partial{L}}{\partial{\textbf{X}}} = \frac{\partial{L}}{\partial{\textbf{Y}}} \cdot \textbf{W}^{\top} $$
$$ \frac{\partial{L}}{\partial{\textbf{W}}} = \textbf{X}^{\top} \cdot \frac{\partial{L}}{\partial{\textbf{Y}}} $$
- 구현 예는 다음과 같다.

In [2]:
import numpy as np


class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 편향의 역전파는 각 데이터마다 구하므로 각 데이터의 역전파 값을 더한다. 
        return dx

### Softmax-with-Loss 
- 소프트맥스 함수는 'Softmax' 계층, 교차 엔트로피 오차는 'Cross Entropy Error' 계층, 이 둘을 조합한 계층을 **Softmax-with-Loss** 계층이라고 한다. 
- 에러 함수는 같다.
### $$ E(\textbf{W}) = -\sum_{k=1}^{K}{d_k} log y_k, y_k = \frac{e^{u_k}}{\sum_{j=1}^Ke^{u_j}} $$

#### 계산 그래프를 이용하는 방법
- 위 식의 역전파는 계산 그래프를 이용해서 구할 수 있으며, 그 방법은 다음과 같다.
  - 가장 오른쪽 (최종 출력 또는 초기값) 역전파의 값은 1이다. ($\frac{\partial{L}}{\partial{L}} = 1$ 이므로)
  - 'x'노드의 역전파는 순전파 시의 입력값을 '서로 바꿔' 상류의 미분에 곱하고 하류로 흘린다.
  - '+'노드에서는 상류에서 전해지는 미분을 그대로 흘린다.
  - 'log'노드의 역전파는 미분한 값을 적용한다. ($y = logx, \frac{\partial{y}}{\partial{x}} = \frac{1}{x}$ )
  - 'exp'노드의 역전파도 미분한 값을 적용한다. ($y = exp(x), \frac{\partial{y}}{\partial{x}} = exp(x)$ )

#### 미분을 이용하는 방법

- 역전파 알고리즘은 단순히 각 레이어의 뉴런에 체인룰을 적용한 것이다.

- 따라서, 체인룰에 따라 **Softmax-with-loss** 계층에 대한 출력 에러 함수의 미분은 다음과 같다.

### $$ \frac{\partial{E}}{\partial{u_j}} = \sum_{j=1}^{K} \frac{\partial{E}}{\partial{y_j}} \cdot \frac{\partial{y_j}}{\partial{u_j}}  $$

- 먼저, 교차 엔트로피의 미분 $ \frac{\partial{E}}{\partial{y_j}} $은 다음과 같다.

### $$ \frac{\partial{E}}{\partial{y_j}} = - \frac{d_j}{y_j} $$ 

- 다음으로, 소프트맥스 함수의 미분 $ \frac{\partial{y_j}}{\partial{u_j}} $ 은 다음과 같다.
- 소프트맥스 함수의 미분은, 분모가 모든 입력신호의 합으로 구성되어 출력층의 각 뉴런이 모든 입력 신호에서 영향을 받기 때문에, $k = j$ 일 때와 $k \neq j$ 인 경우로 두 가지 경우로 미분을 전개한다.

- $k = j$ 일 때,

### $$ \begin{split} \frac{\partial{y_k}}{\partial{u_k}} & = \frac{\sum_j^Ke^{u_j} \cdot e^{u_k} - e^{u_k} \cdot e^{u_k}} {(\sum_j^K{e^{u_j}})^2} \\ & = y_k  - {y_k}^2 \\ & = y_k(1 - y_k)  \end{split} $$

- $k \neq j$ 일 때, 

### $$ \begin{split} \frac{\partial{y_k}}{\partial{u_j}} & = \frac{0 - e^{u_k} \cdot e^{u_j}} {(\sum_j^K{e^{u_j}})^2} \\ & = - y_ky_j \end{split} $$

- 따라서 소프트맥스 함수에 대한 교차 엔트로피의 미분은 다음과 같다:

### $$ \begin{split}  \frac{\partial{E}}{\partial{u_j}} & = - \sum_{k=1}^{K} \frac{\partial{d_klogy_j}}{\partial{u_j}} = - \sum_{j=1}^{K} d_k \frac{\partial{logy_j}}{\partial{u_j}} \\ & = - \frac{d_k}{y_k}\frac{\partial{y_k}}{\partial{u_k}} - \sum_{j \neq k}^{K} \frac{d_j}{y_j} \frac{\partial{y_k}}{\partial{u_j}} \\ & = - \frac{d_k}{y_k} \cdot y_k(1 - y_k) - \sum_{j \neq k}^{K}\frac{d_j}{y_j} \cdot (-y_ky_j) \\ & = - d_k + d_ky_k + \sum_{j \neq k}^{K}d_jy_k \\ & = -d_k + \sum_{j = 1}^{K}d_jy_k \\ & = -d_k + y_k \sum_{j = 1}^{K}d_j \\ & = y_k - d_k  \end{split} $$

- 위 식에서 소프트맥스 함수의 특성상, $ \sum_{j = 1}^{K}d_j = 1 $ 이다.

- 이상, **Softmax-with-loss**의 역전파는 신경망의 츌력 $y_k$와 목표 출력 $d_k$의 차가 된다.  

- **Softmax-with-loss**의 구현 코드는 다음과 같다.

In [3]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

### 오차역전파법 신경망 구현하기

In [4]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *

from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
       
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads        

### 기울기 검증하기
- 수치 미분은 느리지만, 오차역전파법을 제대로 구현했는지 검증하는데 사용한다.

In [5]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:2.75698649643e-13
b1:1.10927381647e-12
W2:1.01970150005e-12
b2:1.19682055932e-10


### 오차역전파 기반 학습 구현하기

In [6]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train_acc: {:0.3f}, test_acc: {:0.3f}".format(train_acc, test_acc))

train_acc: 0.133, test_acc: 0.128
train_acc: 0.905, test_acc: 0.908
train_acc: 0.926, test_acc: 0.928
train_acc: 0.935, test_acc: 0.935
train_acc: 0.945, test_acc: 0.942
train_acc: 0.951, test_acc: 0.947
train_acc: 0.955, test_acc: 0.952
train_acc: 0.959, test_acc: 0.954
train_acc: 0.963, test_acc: 0.958
train_acc: 0.966, test_acc: 0.959
train_acc: 0.969, test_acc: 0.961
train_acc: 0.969, test_acc: 0.961
train_acc: 0.973, test_acc: 0.963
train_acc: 0.975, test_acc: 0.966
train_acc: 0.976, test_acc: 0.967
train_acc: 0.976, test_acc: 0.968
train_acc: 0.977, test_acc: 0.968
